In [1]:
def initial_guess(T,X1,X2,P1sat,P2sat):
    P = (X1*P1sat) + (X2*P2sat)
    Y1 = (X1 * P1sat) / (P)
    Y2 = (X2 * P2sat) / (P)
#    print("Value of Y1 inside the function is :",Y1)
#    print("Value of Y2 inside the function is :",Y2)
#    print("Value of P inside the function is :",P)
    return(P,Y1,Y2)
    
    

In [2]:
def EOS_constant(Tc_1,Pc_1,omega_1,Tc_2,Pc_2,omega_2,T):
    Pc_1 = 1000000 * Pc_1 # Converting pressure in Pa
    Pc_2 = 1000000 * Pc_2 # Converting pressure in Pa
    a1 = (0.45724*8.314*8.314*Tc_1*Tc_1) / Pc_1
    a2 = (0.45724*8.314*8.314*Tc_2*Tc_2) / Pc_2
    b1 = (0.07780*8.314*Tc_1) / Pc_1
    b2 = (0.07780*8.314*Tc_2) / Pc_2
    A1 = 0.37464 + (1.54226*omega_1) - (0.26992*omega_1*omega_1)
    B1 = 1-((T/Tc_1)**0.5)
    alpha1 = (1+(A1*B1))**2
    A2 = 0.37464 + (1.54226*omega_2) - (0.26992*omega_2*omega_2)
    B2 = 1-((T/Tc_2)**0.5)
    alpha2 = (1+(A2*B2))**2
#    print("Value of a1 inside the function is :",a1)
#    print("Value of b1 inside the function is :",b1)
#    print("Value of a2 inside the function is :",a2)
#    print("Value of b2 inside the function is :",b2)
    return(a1,b1,alpha1,a2,b2,alpha2)
    

In [3]:
def mixingrule(a1,b1,alpha1,a2,b2,alpha2,z1,z2,T):
    if T == 291.15:
        k12 = 0.1250
        k21 = 0.1250
        l12 = 0.0514
        l21 = 0.0514
    elif T == 298.17:
        k12 = 0.1053
        k21 = 0.1053
        l12 = 0.0302
        l21 = 0.0302
    elif T == 303.12:
        k12 = 0.1020
        k21 = 0.1020
        l12 = 0.0207
        l21 = 0.0207
    elif T == 308.11:
        k12 = 0.0961
        k21 = 0.0961
        l12 = 0.0083
        l21 = 0.0083
    elif T == 313.14:
        k12 = 0.0944
        k21 = 0.0944
        l12 = 0.0067
        l21 = 0.0067
        
    a11_alpha11 = a1*alpha1
    a22_alpha22 = a2*alpha2
    a12_alpha12 = (1-k12) * ((a1*alpha1*a2*alpha2)**0.5)
    a21_alpha21 = (1-k21) * ((a2*alpha2*a1*alpha1)**0.5)
    a_alpha_mix = (a11_alpha11 * z1 * z1) + (a12_alpha12 * z1 * z2) + (a21_alpha21 * z2 * z1) + (a22_alpha22 * z2 * z2)
    
    b11 = (b1+b1) * 0.5
    b12 = (1-l12) * (b1 + b2) *0.5
    b21 = (1-l21) * (b1 + b2) *0.5
    b22 = (b2 +b2) *0.5
    bmix = (b11 * z1 * z1) + (b12 * z1 * z2) + (b21 * z2 * z1) +(b22 * z2 * z2)
#    print("Value of amix inside the function is :",a_alpha_mix)
#    print("Value of bmix inside the function is :",bmix)
    
    return(a_alpha_mix,bmix,a11_alpha11,a12_alpha12,a21_alpha21,a22_alpha22,b11,b12,b21,b22)

In [4]:
def Vxcal(P,T,a_alpha_mix,bmix,V_y_mix):
    
    Delv = 1
    Vold = V_y_mix / 10 #initial guess
    while Delv>0.001:
        Vnew = bmix + ((8.314*T)/(P+(a_alpha_mix/((Vold*Vold)+(2*bmix*Vold)-(bmix*bmix)))))
        Delv = abs(Vnew - Vold)
        Vold = Vnew
    V = Vold
    print("Value of Volume in function is", V)
    return V

In [5]:
def Vycal(P,T,a_alpha_mix,bmix):
    Delv = 1
    Vold = (8.314*T)/(P) #initial guess
    while Delv > 0.001:
        Vnew = bmix + ((8.314*T)/(P+(a_alpha_mix/((Vold*Vold)+(2*bmix*Vold)-(bmix*bmix)))))
        Delv = abs(Vnew - Vold)
        Vold = Vnew
    V = Vold
    print("Value of Volume in function is", V)
    return V
#V = volumecal(2,3,10,325.15)
#print("Volume is :",V)

In [6]:
def Zcal(P,V,T):
    Z = (P * V)/(8.314 * T)
#    print("Value of Z inside the function is :",Z)
    return(Z)
    

In [7]:
def Phical(a11,a12,a21,a22,b1,b2,bmix,zmix,Vmix,z1,z2,a_alpha_mix): #vmix is V_Lmix and V_Vmix
    import numpy as np
    import math
    import array as arr  
    ln_phi_1 = (b1/bmix)*(zmix-1) - math.log(abs(zmix*(1-(bmix/Vmix)))) + ((a_alpha_mix)/(2*1.414*bmix*8.314*T))*(math.log(abs((Vmix+(2.414*bmix))/(Vmix-(0.414*bmix)))))*((b1/bmix)-(2*((z1*a11)+(z2*a21))/a_alpha_mix))
    ln_phi_2 = (b2/bmix)*(zmix-1) - math.log(abs(zmix*(1-(bmix/Vmix)))) + ((a_alpha_mix)/(2*1.414*bmix*8.314*T))*(math.log(abs((Vmix+(2.414*bmix))/(Vmix-(0.414*bmix)))))*((b2/bmix)-(2*((z1*a12)+(z2*a22))/a_alpha_mix))
#    print("Value of phi_1 inside the function is :",ln_phi_1)
#    print("Value of phi_2 inside the function is :",ln_phi_2)
    phi_1 = math.exp(ln_phi_1)
    phi_2 = math.exp(ln_phi_2)
#    print("Value of phi_1 inside the function is :",phi_1)
#    print("Value of phi_2 inside the function is :",phi_2)
    return(phi_1,phi_2)

In [8]:
def Kcal(phi_1_x,phi_2_x,phi_1_y,phi_2_y):
    K1 = (phi_1_x) / (phi_1_y)
    K2 = (phi_2_x) / (phi_2_y)
#    print("Value of K1 inside the function is :",K1)
#    print("Value of K2 inside the function is :",K2)
    return(K1,K2)

In [9]:
def Ycal(K1,K2,x1,x2):
    y1 = K1 * x1
    y2 = K2 * x2
#    print("Value of y1 inside the function is :",y1)
#    print("Value of y1 inside the function is :",y2)
    return(y1,y2)

In [10]:
def kixi (K1,K2,X1,X2):
    K1x1 = K1 * X1
    K2x2 = K2 * X2
    Sum_Kx = K1x1 + K2x2
    Y1 = (K1x1) / (Sum_Kx)
    Y2 = (K2x2) / (Sum_Kx)
    return(Y1,Y2,Sum_Kx)

In [11]:
def Ynormalize(y1,y2):
    ysum = y1 + y2
    y1 = y1 / ysum
    y2 = y2 / ysum
#    print("Value of y1 inside the function is :",y1)
#    print("Value of y2 inside the function is :",y2)
    return(y1,y2)

In [12]:
if __name__ == "__main__":
    T = float(input("Enter the Temperature of system in K:"))
    n = int(input("Enter number of elements : "))
#    T = 313.14
#   n = 1
    import numpy as np
    import math
    import array as arr
    import random
    print("Enter value of X1")  
    
#    X1 = [0.192,0.289,0.364,0.488,0.587,0.706,0.8,0.834,0.860,0.893,0.924,0.954,0.985]
    X1 = arr.array('f',[])
    X2 = arr.array('f',[])
    y1 = arr.array('f',[])
    y2 = arr.array('f',[])
    Pfinal = arr.array('f',[])
    
    for i in range(0,n):
        x1 = float(input())
        X1.append(x1)
    for i in range(len(X1)):
        x2 = 1 - X1[i]
        X2.append(x2)
    
    # Antoine Equation constant for Carbon-di-oxide (1) from NIST
    A1 = 6.81228
    B1 = 1301.679
    C1 = -3.494
    
    # Antoine Equation constant for Ethanol (2) from NIST
    if T >= 273 and T <= 351.70:
        A2 = 5.37229
        B2 = 1670.409
        C2 = -40.191
    elif T > 351.70 and T <= 366.63:
        A2 = 5.24677
        B2 = 1598.673
        C2 = -46.424
    elif T > 366.63 and T <= 513.9:
        A2 = 4.92531
        B2 = 1432.526
        C2 = -61.819
    
    P1sat = 100000 * (10**((A1)-((B1)/(T + C1)))) # Converting P from bar to Pa
    P2sat = 100000 * (10**((A2)-((B2)/(T + C2)))) #Converting P from bar to Pa
    
    #Critical Constants for Carbon-di-oxide (1) from 4.chiu paper
    Tc_1 = 304.1 # in K
    Pc_1 = 7.38 # in MPa
    omega_1 = 0.225
    
    #Critical Constants for Ethanol (2) from 4.chiu paper
    Tc_2 = 513.9 # in K
    Pc_2 = 6.14 # in MPa
    omega_2 = 0.644 
    
    #Calculating pure component EOS constants
    a1,b1,alpha1,a2,b2,alpha2 = EOS_constant(Tc_1,Pc_1,omega_1,Tc_2,Pc_1,omega_2,T)
    
    #print("Value of a1 is :",a1)
    #print("Value of b1 is :",b1)
    #print("Value of a2 is :",a2)
    #print("Value of b2 is :",b2)
    
    initial_while_loop = 0
    more_while_loop = 0
    less_while_loop = 0
    
    for i in range(0,len(X1)):
        
        P_initial,Y1_initial,Y2_initial = initial_guess(T,X1[i],X2[i],P1sat,P2sat) #step 1 of algorithm strts
        Ysum = Y1_initial + Y2_initial
        print("Initial guess P is:",P_initial)
        print("Roults law Y1 is :",Y1_initial)
        print("Roults law Y2 is :",Y2_initial)
        print("Roults law Ysum is :",Ysum)
        
        Ysum = 0.5
        P = P_initial
        Y1 = Y1_initial
        Y2 = Y2_initial #Step 1 of algorithm ends
        
        #step 2 of algorithm starts
        
        a_alpha_mix_x,bmix_x,a11x,a12x,a21x,a22x,b11x,b12x,b21x,b22x = mixingrule(a1,b1,alpha1,a2,b2,alpha2,X1[i],X2[i],T)
        a_alpha_mix_y,bmix_y,a11y,a12y,a21y,a22y,b11y,b12y,b21y,b22y = mixingrule(a1,b1,alpha1,a2,b2,alpha2,Y1,Y2,T)
                
        #print("Value of a_alpha_mix_x is :",a_alpha_mix_x)
        #print("Value of bmix_x is :",bmix_x)
        #print("Value of a_alpha_mix_y is :",a_alpha_mix_y)
        #print("Value of bmix_y is :",bmix_y)
                
        V_y_mix = Vycal(P,T,a_alpha_mix_y,bmix_y)
        V_x_mix = Vxcal(P,T,a_alpha_mix_x,bmix_x,V_y_mix)
                
        #print("Value of V_y_mix is :",V_y_mix)
        #print("Value of V_x_mix is :",V_x_mix)
                
        Z_x = Zcal(P,V_x_mix,T)
        Z_y = Zcal(P,V_y_mix,T)
                
        #print("Value of Z_x is :",Z_x)
        #print("Value of Z_y is :",Z_y)
                
        phi_1_x,phi_2_x = Phical(a11x,a12x,a21x,a22x,b1,b2,bmix_x,Z_x,V_x_mix,X1[i],X2[i],a_alpha_mix_x)
        phi_1_y,phi_2_y = Phical(a11y,a12y,a21y,a22y,b1,b2,bmix_y,Z_y,V_y_mix,Y1,Y2,a_alpha_mix_y)
                
        #print("Value of phi1 in liquid is :",phi_1_x)
        #print("Value of phi2 in liquid is :",phi_2_x)
        #print("Value of phi1 in vapor is :",phi_1_y)
        #print("Value of phi2 in vapor is :",phi_2_y)
                
        K1,K2 = Kcal(phi_1_x,phi_2_x,phi_1_y,phi_2_y)
                
        Y1,Y2,Sum_Kx_old = kixi(K1,K2,X1[i],X2[i])
        print("K1 initial is :",K1)
        print("K2 initial is :",K2)
        print("Y1 initial is :",Y1)
        print("Y2 initial is :",Y2)
        
        diff = 0.5 # diff in summation kixi
        
        #Step 2 of algorithm ends
        
        while diff > 0.001:
            
            print("Here while of initial guess stats")
            
            a_alpha_mix_x,bmix_x,a11x,a12x,a21x,a22x,b11x,b12x,b21x,b22x = mixingrule(a1,b1,alpha1,a2,b2,alpha2,X1[i],X2[i],T)
            a_alpha_mix_y,bmix_y,a11y,a12y,a21y,a22y,b11y,b12y,b21y,b22y = mixingrule(a1,b1,alpha1,a2,b2,alpha2,Y1,Y2,T)
                
            #print("Value of a_alpha_mix_x is :",a_alpha_mix_x)
            #print("Value of bmix_x is :",bmix_x)
            #print("Value of a_alpha_mix_y is :",a_alpha_mix_y)
            #print("Value of bmix_y is :",bmix_y)
                
            V_y_mix = Vycal(P,T,a_alpha_mix_y,bmix_y)
            V_x_mix = Vxcal(P,T,a_alpha_mix_x,bmix_x,V_y_mix)
                
            #print("Value of V_y_mix is :",V_y_mix)
            #print("Value of V_x_mix is :",V_x_mix)
                
            Z_x = Zcal(P,V_x_mix,T)
            Z_y = Zcal(P,V_y_mix,T)
                
            #print("Value of Z_x is :",Z_x)
            #print("Value of Z_y is :",Z_y)
                
            phi_1_x,phi_2_x = Phical(a11x,a12x,a21x,a22x,b1,b2,bmix_x,Z_x,V_x_mix,X1[i],X2[i],a_alpha_mix_x)
            phi_1_y,phi_2_y = Phical(a11y,a12y,a21y,a22y,b1,b2,bmix_y,Z_y,V_y_mix,Y1,Y2,a_alpha_mix_y)
                
            #print("Value of phi1 in liquid is :",phi_1_x)
            #print("Value of phi2 in liquid is :",phi_2_x)
            #print("Value of phi1 in vapor is :",phi_1_y)
            #print("Value of phi2 in vapor is :",phi_2_y)
                
            K1,K2 = Kcal(phi_1_x,phi_2_x,phi_1_y,phi_2_y)
            
            Y1,Y2,Sum_Kx_new = kixi(K1,K2,X1[i],X2[i])
                
            diff = abs(Sum_Kx_new - Sum_Kx_old)
            print ("diff in initial while loop is :",diff)
            
            Sum_Kx_old = Sum_Kx_new
            initial_while_loop = initial_while_loop + 1
            print("Initial while loop K1 value is :",K1)
            print("Initial while loop K2 value is :",K2)
            print("Initial while loop Y1 value is :",Y1)
            print("Initial while loop Y2 value is :",Y2)
            print("Initial while loop Pressure value is :",Pfinal)
            print("Initial while loop vaue is :",initial_while_loop)
            print("Initial while loop Sum_Kx_old value is :",Sum_Kx_old)
            print("Initial while loop Sum_Kx_new value is :",Sum_Kx_new)
                
            
        if Sum_Kx_new > 1:
                
            print("We aew in Sum_Kx_new > 1 loop")
            print("Initial while loop Sum_Kx_new value is :",Sum_Kx_new)
                
            P = P * Sum_Kx_new
            Y1 = (X1[i] * P1sat) / (P)
            Y2 = (X2[i] * P2sat) / (P)
                
            a_alpha_mix_x,bmix_x,a11x,a12x,a21x,a22x,b11x,b12x,b21x,b22x = mixingrule(a1,b1,alpha1,a2,b2,alpha2,X1[i],X2[i],T)
            a_alpha_mix_y,bmix_y,a11y,a12y,a21y,a22y,b11y,b12y,b21y,b22y = mixingrule(a1,b1,alpha1,a2,b2,alpha2,Y1,Y2,T)
                
            #print("Value of a_alpha_mix_x is :",a_alpha_mix_x)
            #print("Value of bmix_x is :",bmix_x)
            #print("Value of a_alpha_mix_y is :",a_alpha_mix_y)
            #print("Value of bmix_y is :",bmix_y)
                
            V_y_mix = Vycal(P,T,a_alpha_mix_y,bmix_y)
            V_x_mix = Vxcal(P,T,a_alpha_mix_x,bmix_x,V_y_mix)
                
            #print("Value of V_y_mix is :",V_y_mix)
            #print("Value of V_x_mix is :",V_x_mix)
                
            Z_x = Zcal(P,V_x_mix,T)
            Z_y = Zcal(P,V_y_mix,T)
                
            #print("Value of Z_x is :",Z_x)
            #print("Value of Z_y is :",Z_y)
                
            phi_1_x,phi_2_x = Phical(a11x,a12x,a21x,a22x,b1,b2,bmix_x,Z_x,V_x_mix,X1[i],X2[i],a_alpha_mix_x)
            phi_1_y,phi_2_y = Phical(a11y,a12y,a21y,a22y,b1,b2,bmix_y,Z_y,V_y_mix,Y1,Y2,a_alpha_mix_y)
                
            #print("Value of phi1 in liquid is :",phi_1_x)
            #print("Value of phi2 in liquid is :",phi_2_x)
            #print("Value of phi1 in vapor is :",phi_1_y)
            #print("Value of phi2 in vapor is :",phi_2_y)
                
            K1,K2 = Kcal(phi_1_x,phi_2_x,phi_1_y,phi_2_y)
                
            Y1,Y2,Sum_Kx_old = kixi(K1,K2,X1[i],X2[i])
            diff = 0.5
                
            while diff > 0.001:
                    
                print("Here while loop of Sum_Kx_new > 1 starts")
                    
                a_alpha_mix_x,bmix_x,a11x,a12x,a21x,a22x,b11x,b12x,b21x,b22x = mixingrule(a1,b1,alpha1,a2,b2,alpha2,X1[i],X2[i],T)
                a_alpha_mix_y,bmix_y,a11y,a12y,a21y,a22y,b11y,b12y,b21y,b22y = mixingrule(a1,b1,alpha1,a2,b2,alpha2,Y1,Y2,T)
                    
                #print("Value of a_alpha_mix_x is :",a_alpha_mix_x)
                #print("Value of bmix_x is :",bmix_x)
                #print("Value of a_alpha_mix_y is :",a_alpha_mix_y)
                #print("Value of bmix_y is :",bmix_y)
                    
                V_y_mix = Vycal(P,T,a_alpha_mix_y,bmix_y)
                V_x_mix = Vxcal(P,T,a_alpha_mix_x,bmix_x,V_y_mix)
                    
                #print("Value of V_y_mix is :",V_y_mix)
                #print("Value of V_x_mix is :",V_x_mix)
                    
                Z_x = Zcal(P,V_x_mix,T)
                Z_y = Zcal(P,V_y_mix,T)
                    
                #print("Value of Z_x is :",Z_x)
                #print("Value of Z_y is :",Z_y)
                    
                phi_1_x,phi_2_x = Phical(a11x,a12x,a21x,a22x,b1,b2,bmix_x,Z_x,V_x_mix,X1[i],X2[i],a_alpha_mix_x)
                phi_1_y,phi_2_y = Phical(a11y,a12y,a21y,a22y,b1,b2,bmix_y,Z_y,V_y_mix,Y1,Y2,a_alpha_mix_y)
                    
                #print("Value of phi1 in liquid is :",phi_1_x)
                #print("Value of phi2 in liquid is :",phi_2_x)
                #print("Value of phi1 in vapor is :",phi_1_y)
                #print("Value of phi2 in vapor is :",phi_2_y)
                    
                K1,K2 = Kcal(phi_1_x,phi_2_x,phi_1_y,phi_2_y)
                    
                Y1,Y2,Sum_Kx_new = kixi(K1,K2,X1[i],X2[i])
                    
                diff = abs(Sum_Kx_new - Sum_Kx_old)
                print ("diff in while loop of Sum_Kx_new > 1 is :",diff)
                    
                Sum_Kx_old = Sum_Kx_new
                more_while_loop = more_while_loop + 1
                Pfinal = P / 10000000 # Convertision from Pato Mpa
                print(">>>>K1 in while loop of Sum_Kx_new > 1 is >>>>:",K1)
                print(">>>>K2 in while loop of Sum_Kx_new > 1 is >>>>:",K2)
                print(">>>>Y1 in while loop of Sum_Kx_new > 1 is >>>>:",Y1)
                print(">>>>Y2 in while loop of Sum_Kx_new > 1 is >>>>:",Y2)
                print(">>>>P in while loop of Sum_Kx_new > 1 is >>>>:",Pfinal)
                print(">>>>more_while_loop in while loop of Sum_Kx_new > 1 is >>>>:",more_while_loop)
                print(">>>>Sum_Kx_old in while loop of Sum_Kx_new > 1 is >>>>:",Sum_Kx_old)
                print(">>>>Sum_Kx_new in while loop of Sum_Kx_new > 1 is >>>>:",Sum_Kx_new)
                    
        if Sum_Kx_new < 1:
            
            print("We aew in Sum_Kx_new < 1 loop")
            print("Initial while loop Sum_Kx_new value is :",Sum_Kx_new)
                
            P = P * Sum_Kx_new
            Y1 = (X1[i] * P1sat) / (P)
            Y2 = (X2[i] * P2sat) / (P)
                
            a_alpha_mix_x,bmix_x,a11x,a12x,a21x,a22x,b11x,b12x,b21x,b22x = mixingrule(a1,b1,alpha1,a2,b2,alpha2,X1[i],X2[i],T)
            a_alpha_mix_y,bmix_y,a11y,a12y,a21y,a22y,b11y,b12y,b21y,b22y = mixingrule(a1,b1,alpha1,a2,b2,alpha2,Y1,Y2,T)
                
            #print("Value of a_alpha_mix_x is :",a_alpha_mix_x)
            #print("Value of bmix_x is :",bmix_x)
            #print("Value of a_alpha_mix_y is :",a_alpha_mix_y)
            #print("Value of bmix_y is :",bmix_y)
                
            V_y_mix = Vycal(P,T,a_alpha_mix_y,bmix_y)
            V_x_mix = Vxcal(P,T,a_alpha_mix_x,bmix_x,V_y_mix)
                
            #print("Value of V_y_mix is :",V_y_mix)
            #print("Value of V_x_mix is :",V_x_mix)
                
            Z_x = Zcal(P,V_x_mix,T)
            Z_y = Zcal(P,V_y_mix,T)
                
            #print("Value of Z_x is :",Z_x)
            #print("Value of Z_y is :",Z_y)
                
            phi_1_x,phi_2_x = Phical(a11x,a12x,a21x,a22x,b1,b2,bmix_x,Z_x,V_x_mix,X1[i],X2[i],a_alpha_mix_x)
            phi_1_y,phi_2_y = Phical(a11y,a12y,a21y,a22y,b1,b2,bmix_y,Z_y,V_y_mix,Y1,Y2,a_alpha_mix_y)
                
            #print("Value of phi1 in liquid is :",phi_1_x)
            #print("Value of phi2 in liquid is :",phi_2_x)
            #print("Value of phi1 in vapor is :",phi_1_y)
            #print("Value of phi2 in vapor is :",phi_2_y)
                
            K1,K2 = Kcal(phi_1_x,phi_2_x,phi_1_y,phi_2_y)
                
            Y1,Y2,Sum_Kx_old = kixi(K1,K2,X1[i],X2[i])
            diff = 0.5
                
            while diff > 0.001:
                    
                print("Here while loop of Sum_Kx_new < 1 starts : ")
                    
                a_alpha_mix_x,bmix_x,a11x,a12x,a21x,a22x,b11x,b12x,b21x,b22x = mixingrule(a1,b1,alpha1,a2,b2,alpha2,X1[i],X2[i],T)
                a_alpha_mix_y,bmix_y,a11y,a12y,a21y,a22y,b11y,b12y,b21y,b22y = mixingrule(a1,b1,alpha1,a2,b2,alpha2,Y1,Y2,T)
                    
                #print("Value of a_alpha_mix_x is :",a_alpha_mix_x)
                #print("Value of bmix_x is :",bmix_x)
                #print("Value of a_alpha_mix_y is :",a_alpha_mix_y)
                #print("Value of bmix_y is :",bmix_y)
                    
                V_y_mix = Vycal(P,T,a_alpha_mix_y,bmix_y)
                V_x_mix = Vxcal(P,T,a_alpha_mix_x,bmix_x,V_y_mix)
                    
                #print("Value of V_y_mix is :",V_y_mix)
                #print("Value of V_x_mix is :",V_x_mix)
                    
                Z_x = Zcal(P,V_x_mix,T)
                Z_y = Zcal(P,V_y_mix,T)
                    
                #print("Value of Z_x is :",Z_x)
                #print("Value of Z_y is :",Z_y)
                    
                phi_1_x,phi_2_x = Phical(a11x,a12x,a21x,a22x,b1,b2,bmix_x,Z_x,V_x_mix,X1[i],X2[i],a_alpha_mix_x)
                phi_1_y,phi_2_y = Phical(a11y,a12y,a21y,a22y,b1,b2,bmix_y,Z_y,V_y_mix,Y1,Y2,a_alpha_mix_y)
                    
                #print("Value of phi1 in liquid is :",phi_1_x)
                #print("Value of phi2 in liquid is :",phi_2_x)
                #print("Value of phi1 in vapor is :",phi_1_y)
                #print("Value of phi2 in vapor is :",phi_2_y)
                    
                K1,K2 = Kcal(phi_1_x,phi_2_x,phi_1_y,phi_2_y)
                    
                Y1,Y2,Sum_Kx_new = kixi(K1,K2,X1[i],X2[i])
                    
                diff = abs(Sum_Kx_new - Sum_Kx_old)
                print ("diff after is :",diff)
                    
                Sum_Kx_old = Sum_Kx_new
                less_while_loop = less_while_loop + 1
                Pfinal = P / 10000000 # Convertision from Pa to Mpa
                print("<<<<<K1 in while loop of Sum_Kx_new < 1 is <<<<<:",K1)
                print("<<<<<K2 in while loop of Sum_Kx_new < 1 is <<<<<:",K2)
                print("<<<<<Y1 in while loop of Sum_Kx_new < 1 is <<<<<:",Y1)
                print("<<<<<Y2 in while loop of Sum_Kx_new < 1 is <<<<<:",Y2)
                print("<<<<<P in while loop of Sum_Kx_new < 1 is <<<<<:",Pfinal)
                print("<<<<<less_while_loop in while loop of Sum_Kx_new < 1 is <<<<<:",less_while_loop)
                    
            
        if Sum_Kx_new == 1:
            Pfinal = P / 10000000 # Convertision from Pa to Mpa
            print("We aew in Sum_Kx_new = 1 loop")
            print("Initial while loop Sum_Kx_new value is :",Sum_Kx_new)
            print("Correct P is :", Pfinal)
            print("Correct Y1 is :",Y1)
            print("Correct Y2 is :",Y2)

            
                    
            
    
    
    

Enter value of X1
0.985
Initial guess P is: 39990257.94400179
Roults law Y1 is : 0.9999932923702285
Roults law Y2 is : 6.707629771525295e-06
Roults law Ysum is : 1.0
Value of Volume in function is 5.3941296064932896e-05
Value of Volume in function is 2.4167789231739293e-05
K1 initial is : 1.0462898385321655
K2 initial is : 0.07219504788567033
Y1 initial is : 0.9989503272983138
Y2 initial is : 0.0010496727016861768
Here while of initial guess stats
Value of Volume in function is 5.393107311495488e-05
Value of Volume in function is 2.4167321197644637e-05
diff in initial while loop is : 0.0001578562563562258
Initial while loop K1 value is : 1.0464437238520785
Initial while loop K2 value is : 0.07261366255427858
Initial while loop Y1 value is : 0.9989444024081782
Initial while loop Y2 value is : 0.0010555975918218876
Initial while loop Pressure value is : array('f')
Initial while loop vaue is : 1
Initial while loop Sum_Kx_old value is : 1.0318362868633622
Initial while loop Sum_Kx_new valu